<a href="https://colab.research.google.com/github/s-mostafa-a/lung-cancer-segmentation/blob/master/using_wolny_package_for_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def _cn_t_d():
  from google.colab import drive
  drive.mount("/content/drive")

In [2]:
def _untar_data():
  !tar -xf ./drive/My\ Drive/Copy_Task06_Lung.tar -C ./

In [3]:
def connect_to_drive():
  !mkdir drive
  _cn_t_d()
  _untar_data()

In [4]:
def install_all_needed_packages():
  !pip install git+git://github.com/s-mostafa-a/niftidataset.git
  !git clone https://github.com/wolny/pytorch-3dunet.git
  !pip install hdbscan

In [14]:
def make_databunch():
  np.random.seed(1)
  train_dir = '/content/Task06_Lung/'
  patch_sz = 110
  tfms = tvt.Compose([RandomCrop3D(patch_sz), ToTensor(), Normalize(mean=[0],std=[1],is_3d=True)])
  tds, vds = get_train_and_validation_from_one_directory(source_dir=train_dir+'imagesTr', target_dir=train_dir+'labelsTr', valid_pct=0.2, transform=tfms)
  print(len(tds), len(vds))
  idb = faiv.ImageDataBunch.create(tds, vds, bs=1, num_workers=1)
  return idb

In [19]:
def make_learner(idb):
  model = UNet3D(in_channels=1, out_channels=1, final_sigmoid=True)
  metric = metrics.DiceCoefficient()
  metric.__name__ = 'DiceCoefficient'
  learner = faiv.Learner(idb, model, loss_func=losses.BCEDiceLoss(alpha=0.3, beta=0.7), metrics=[metric], opt_func=torch.optim.Adam)
  return learner


In [ ]:
# connect_to_drive()
# install_all_needed_packages()


#imports
import numpy as np
import os
import sys
module_path = os.path.abspath(os.path.join('/content/pytorch-3dunet/'))
if module_path not in sys.path:
    sys.path.append(module_path)
from pytorch3dunet.unet3d.model import UNet3D, ResidualUNet3D
from pytorch3dunet.unet3d import losses
from pytorch3dunet.unet3d import metrics
from niftidataset import get_train_and_validation_from_one_directory, RandomCrop3D, ToTensor, Normalize
from torchvision import transforms as tvt
from torch.nn import BCEWithLogitsLoss
import torch 
from fastai import vision as faiv
%matplotlib inline
%load_ext autoreload
%autoreload 2
!export CUDA_LAUNCH_BLOCKING=1
#imports/


idb = make_databunch()
learner = make_learner(idb)


learner.lr_find(start_lr=1e-10, end_lr=10)
learner.recorder.plot()
# found that 1e+00 works fine

**Loading the model from drive:**

In [21]:
model_name = '3d_lung_seg_model'

In [22]:
!rm -r ./models
!mkdir ./models
!cp ./drive/My\ Drive/3d_lung_seg_model.pth ./models/3d_lung_seg_model.pth 

In [ ]:
learner.load(model_name)

In [ ]:
learner.fit_one_cycle(10, max_lr=slice(1e-01), pct_start=0.5)

epoch,train_loss,valid_loss,DiceCoefficient,time


As mentioned in the paper [Tversky loss function for image segmentation using 3D fully convolutional deep networks](https://arxiv.org/pdf/1706.05721.pdf), training process takes about 1000 epochs and best result is when beta equals to 0.7 (and alpha is 0.3).

**Saving model to drive:**

In [ ]:
learner.save(model_name)
learner.load(model_name)

In [ ]:
!cp ./models/3d_lung_seg_model.pth ./drive/My\ Drive/3d_lung_seg_model.pth